In [8]:
import os
import firebase_admin
from firebase_admin import credentials, db
import pandas as pd

In [9]:
# ✅ Get Firebase credentials path
cred_path = os.getenv("FIREBASE_KEY_PATH")

if not cred_path:
    raise ValueError("FIREBASE_KEY_PATH is not set. Make sure the environment variable is configured.")

# ✅ Define database URL
DATABASE_URL = "https://umsports-linksync-default-rtdb.europe-west1.firebasedatabase.app/"

# ✅ Print debug info
print("🔍 Using Firebase Key Path:", cred_path)
print("🔍 Using Database URL:", DATABASE_URL)

# ✅ Initialize Firebase if not already initialized
if not firebase_admin._apps:
    cred = credentials.Certificate(cred_path)
    firebase_admin.initialize_app(cred, {'databaseURL': DATABASE_URL})

# ✅ Check if databaseURL is set correctly
app = firebase_admin.get_app()
print("✅ Firebase initialized with options:", app.options)

# ✅ Explicitly print databaseURL
print("🔍 Confirmed databaseURL in Firebase app:", app.options.get("databaseURL"))

# ✅ Connect to Realtime Database
ref = db.reference("/")  # Root of the database

# ✅ Test writing data
ref.child("test").set({
    "message": "Realtime Database is working!"
})

print("✅ Realtime Database connection successful! Check Firebase Console for 'test' data.")

🔍 Using Firebase Key Path: C:\Users\lara\Documents\firebase_key.json
🔍 Using Database URL: https://umsports-linksync-default-rtdb.europe-west1.firebasedatabase.app/
✅ Firebase initialized with options: <firebase_admin._AppOptions object at 0x0000015C7F4C04D0>
🔍 Confirmed databaseURL in Firebase app: https://umsports-linksync-default-rtdb.europe-west1.firebasedatabase.app/
✅ Realtime Database connection successful! Check Firebase Console for 'test' data.


In [3]:
def upload_users_to_firebase():
    file_path = "users.xlsx"  # Ensure this file is in the correct path
    users_df = pd.read_excel(file_path)

    users_ref = db.reference("users")  # Reference to Firebase "users" node

    for _, row in users_df.iterrows():
        username = row["Username"]
        password = row["Password"]
        name = row["Name"]

        # Add user data to Firebase
        users_ref.child(username).set({
            "password": password,
            "name": name
        })

    print("✅ All users uploaded to Firebase successfully!")

# Run the function to upload users
upload_users_to_firebase()

✅ All users uploaded to Firebase successfully!
